In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import pandas as pd
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('coords.csv')

In [3]:
X = df.drop('class', axis = 1)
y = df['class']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42, stratify = y)

In [5]:
models = {
    'lr': make_pipeline(StandardScaler(), LogisticRegression(max_iter = 300)),
    'rc': make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gbc': make_pipeline(StandardScaler(), GradientBoostingClassifier())
}

fit_models = {}

for algo, model in models.items():
    fit_models[algo] = model.fit(X_train, y_train)

In [6]:
for algo, model in fit_models.items():
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    print(algo, acc)

lr 0.9855902777777777
rc 0.9817708333333334
gbc 0.9623263888888889


## Neural Network

In [8]:
import string
letters = list(string.ascii_uppercase)
letters

['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z']

In [9]:
numbers = [str(num) for num in range(0, 10)]
numbers

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [10]:
class_names = letters + numbers
class_names

['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9']

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import pandas as pd
from sklearn.metrics import accuracy_score

In [12]:
df = pd.read_csv('coords.csv')
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z19,v19,x20,y20,z20,v20,x21,y21,z21,v21
0,A,0.596015,0.598637,-5.351079e-07,0.0,0.524824,0.590247,0.003737,0.0,0.483084,...,0.008444,0.0,0.570060,0.442771,0.020445,0.0,0.584027,0.460907,0.033485,0.0
1,A,0.582702,0.602976,-4.789155e-07,0.0,0.514781,0.599014,0.001139,0.0,0.469273,...,0.004532,0.0,0.555620,0.450476,0.013084,0.0,0.570349,0.470588,0.025043,0.0
2,A,0.568719,0.601903,-4.693277e-07,0.0,0.502747,0.596997,0.000186,0.0,0.459650,...,-0.004795,0.0,0.544712,0.446647,0.002810,0.0,0.558533,0.469868,0.014419,0.0
3,A,0.559268,0.600854,-4.881787e-07,0.0,0.491037,0.593645,0.006739,0.0,0.447823,...,0.004258,0.0,0.532012,0.447075,0.011150,0.0,0.547573,0.469695,0.022285,0.0
4,A,0.548125,0.613788,-4.889142e-07,0.0,0.476834,0.605301,0.008348,0.0,0.434377,...,0.007032,0.0,0.514065,0.463542,0.014416,0.0,0.530254,0.481641,0.025965,0.0


In [13]:
df.dropna(axis = 0, inplace = True)

In [14]:
X = df.drop('class', axis = 1)
y = df['class']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42, stratify = y)

In [16]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import InputLayer, Dense

/Users/juancarlostapiabaeza/Desktop/Projects/env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [17]:
def create_asl_nn():
  model = Sequential([
      Dense(input_shape=(84,), units=84, activation='relu'),
      Dense(units=32, activation='relu'),
      Dense(units=16, activation='relu'),
      Dense(units=len(class_names), activation='softmax')
  ])
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
  return model

In [28]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from tensorflow.keras.callbacks import EarlyStopping

trans = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
clf = KerasClassifier(build_fn = create_asl_nn, epochs=100, batch_size=200, shuffle=True, verbose=1, validation_split=0.2 ,callbacks=[EarlyStopping(min_delta=0.001, patience=5)])
pipe = make_pipeline(StandardScaler(), clf)

In [29]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_train_n = label_encoder.fit_transform(y_train)
pipe.fit(X_train, y_train_n)

/Users/juancarlostapiabaeza/Desktop/Projects/env/lib/python3.9/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


AttributeError: 'Sequential' object has no attribute 'compiled'

In [ ]:
preds = clf.predict(X_test)
preds = label_encoder.inverse_transform(y=preds)
acc = accuracy_score(y_test, preds)
print('NN', acc)

In [ ]:
label_encoder.inverse_transform(clf.predict(pd.DataFrame([X_test.iloc[0][:]])))[0]

In [ ]:
max(clf.predict_proba(pd.DataFrame([X_test.iloc[0][:]])))

In [ ]:
# Save model
clf.save("neural_network.h5")